In [8]:
import numpy as np
import matplotlib as py


# Word 2 Vec: Introduction
The main premise behind Word 2 Vec is to utilize the theory of meaning
of words, with aim to establish a coherent embedding between every word and it’s associated context
words. The algorithm written by (FIX: Mik et.all) has two different schematics
associated with it:
    1. Skip Gram(SG) - Predict context words given a target word (position independant)
    2. Continous Bag of Words(CBOW) - Predict one word from context words associated with a single target word
In this summary, we will focus on the Skip gram model (SGM) with aim to give the reader insight of the derivation of the method, how the method is used in practice, and an outline of the code associated with it.

Under the SGM, the training objective is to find a coherent vector representation for every word in the corpus and embed these vectors in $\mathbb{R}^{d}$ with $d$ being the user specified dimension the of the embedding. For each training step, a word in the corpus is selected as the target word $w_{t}$ and is surrounded by a sample window size $2m+1$ where $m$ is the number of context words on each side of a given target word. These context words are words that immediately precede and follow a target word to establish a meaning of a given target word. Thus, it is expected that in the emebedded space words of similar meaning will be closer together, while words that are dissimilar will be further. 
![alt text](window_expl.png "Example ") 
In the example above, we note that our target word is "Banking" and is surrounded by two context words on either side of the target. Of a given corpus, this window will be our training sample for one step of our model. In theory, we could embed all words of a given corpus in a space $\mathbb{R}^{V}$ where $V$ is the number of words in our vocabulary using a **one hot vector** representation for a given word. We recall that the **one hot vector** is represented as a $\mathbb{R}^{V\times 1}$ vector with all 0's besides a 1 at the index of the word we wish to represent. In this space, each word is equidistant thus there is no notion of distance between each word vector and in this space not much relevant information can be taken. Thus, it makes sense to embed these words in a dimension $d<<V$ with intent to "squeeze" the word vectors and place similar words closer to one another and dissimilar further apart.

## Skip Gram

To best understand the SGM, we will begin by analyzing a simple toy example applied to a sentence with a small vocabulary. Like most neural networks the SGM has two phases associated with it, first the feed forward step and second back propogation. The feed forward model is used to calculate the weights of a given target word associated with it's context words, and then back propogate residuals to update the weights at each step.

Before explaining the steps of the model, we will first define a few more parameters used in our toy example. The input to hidden layer matrix $\mathbf{W}$ is the embedding matrix of our corpus, which is initialized randomly between values of 0 and 1. Each column of $\mathbf{W}$ corresponds to the placement of a given word in our corpus, essentially where that word "lives" in our target space $\mathbb{R}^{d}$. Another important parameter is $\mathbf{h}$ which is our hidden layer of the model which will be explained in detail shortly. Matrix $\mathbf{W'}$ is the "context word prediction" matrix which stores the values that will be used to update the probabilities for a given target word and associated context words.

In our toy example, we set the size of the sample window to be three thus there should be two output vectors since # of Vectors = Sample size -1. We denote these vectors as "panels" of the output layer. We label the entire output layer of the model to be $\mathbf{O}$, and if we wish to access a specific element for a certain panel we use the notation $\mathbf{o}_{c,j}$ where $c$ is the panel identification number and $j$ is the element of that panel. Thus for our toy example, $c= 1\ldots C$ where $C$ is the number of panels (in this example two) and $j=1\ldots V$ (for our example $V=5$). We note that there is a "ground truth" vector $\mathbf{w}_{A,c}$ for each context word that will be used to calculate the resilduals and update the weights of $\mathbf{W}$ and $\mathbf{W'}$ via back propagation. We note that the subscript $A$ corresponds to "actual" and $c$ the panel number of the output layer $\mathbf{O}$. To further clarify, $\mathbf{W}$ and $\mathbf{W'}$ are seperate matricies that do not share the same element values. Under each step there is a letter in black font with an arrow above it that will label the step for the model. Each step will now be explained in further detail.
![alt text](Skipgrammodel.png "Example ") 

**A** --> The target word for our vocabulary in a given step is $\mathbf{p}_{t}$. Thus our **one hot vector** is initialized in the following manner such that given our vector $\mathbf{p}$, we set $\mathbf{p}_{t}=1$ and all other $\mathbf{p}_{t'}=0$ for $t\neq t'$ . The role of this vector is to be part of an extraction of the representation of our target word in matrix $\mathbf{W}$. We note that the length of this vector (in the blue box above it's label) is equivalent to the number of words in our vocabulary. 

**B** --> Matrix $\mathbf{W}$ stores the coordinates of our embedding for every word in the vocabulary. The numerical values shown in the matrix correspond to the values of the target word. The dashed lines represent values for the rest of the vocabulary, which for $\mathbf{W}$ is not important since we are only trying to extract the coordinates of our target word. The dimension of $\mathbf{W}$ depeneds on two things, first the length of vocabulary $V$ and second the dimension of the embedding $d$. We recall that the dimension of the embedding is a hyperparameter specified by the user, and is equivalent to the number of neurons in hidden layer $\mathbf{h}$.

**C** --> This is the hidden layer of the matrix $\mathbf{h}$ which acts as a "look up table" that retrives the coordinates of our embedding for the target word $\mathbf{p}_{t}$. This is due to the matrix multiplcation between the **one hot vector** $\mathbf{p}$ and input weight matrix $\mathbf{W}$. Thus the hidden layer of the model $\mathbf{h}$ is defined as $\mathbf{h}= \mathbf{W}\mathbf{p}$ where a single neuron $i$ becomes $\mathbf{h}_{i} = \sum_{t=1}^{V}\mathbf{p}_{t}\cdot\mathbf{W}_{t,i}$ resulting in a vector of dimension $d\times 1$. As expected, the number of rows of this matrix corresponds to the number of neurons in our hidden layer. 

**D** --> Once the hidden layer $\mathbf{h}$ is computed, the model proceeds to compute a matrix product between our hidden layer $\mathbf{h}$ and output weight matrix $\mathbf{W'}$. We note that the same hidden layer > output weight $\mathbf{W'}$ matrix is used for computing each of the panels in our output layer, since there is only one representation for our context words. Thus, we denote our output layer as $\mathbf{O}= \mathbf{W'}^{T}(\mathbf{h}) = \mathbf{W'}^{T}(\mathbf{W}\mathbf{p})$. Since our context matrix representation $\mathbf{W'}$ is the same, each of the panels for the output layer will have identical weight values. This can be seen by observing the two panels $\mathbf{o}_{1}$ and $\mathbf{o}_{2}$. Now, we label the elements of each output layer as $j$, where $j=1$ would be the first element of either of our output panels.  

**E** --> A standard map to convert from $\mathbb{R}^{d}$ to a probability distribution is to use the softmax function $$\mathbb{P}_{i} = \frac{\exp(\mathbf{x}_{i})}{\sum_{j}^{V}\exp (\mathbf{x}_{j})}$$

Applying this map to our output panels resembles the actions of a max function. Since the exponential of elements with a greater magnitude will get larger, they will inherentily dominate the terms of smaller magnitude. Following a normalization across all element of $j$, each panel of our output layer will be similar to that of a probability distribution. Now, using this softmax function in the context of our model gives $$\mathbb{P}(\mathbf{o}_{c,j} = \mathbf{w}_{A} | \mathbf{p}_{t}) = \mathbf{y}_{c,j} = \frac{\exp(\mathbf{v'}_{j}\mathbf{h})}{\sum_{j'}^{V}\exp(\mathbf{v'}_{j'}\mathbf{h})}$$

where
- $\mathbf{v'}_{j}\mathbf{h} = \mathbf{o}_{c,j}$ => j-th word on c-th panel of the output layer
- $\mathbf{w}_{A}$ => Actual c-th word in the output context
- $\mathbf{p}_{t}$ => Target word
- $\mathbf{v'}_{j}$ => Column vector of j-th word from matrix $\mathbf{W'}$
- $\mathbf{h}$ => Hidden layer 

All terms in the above equation are self explanatory, except the arguments in the exponential. For the numerator term, we note that this will be the $j-th$ word probability. Thus $\mathbf{o}_{c,j} = \mathbf{o}_{j} = \mathbf{v'}_{j}^{T}\mathbf{h}$, for $c = 1\ldots C$. We note that this computation will output a scalar value. The denominator term is quite similar, though we now take a sum of the product between $\mathbf{v'}_{j}^{T}\mathbf{h}$ for all columns in $\mathbf{W'}$.

**F** --> This step is focussed around the back propogation aspect of the model, used to update our weight matricies $\mathbf{W}$ and $\mathbf{W'}$. We focus now on the parameter update equation. The loss function is defined as

$$J = -\log(\mathbf{w}_{A,1},\mathbf{w}_{A,2}, \ldots , \mathbf{w}_{A,C}| \mathbf{w}_{t} )$$ $$J =-\log\prod_{c=1}^{C}\frac{\exp(\mathbf{o}_{c,j*})}{\sum_{j'}^{V}\exp(\mathbf{v'}_{j'}\mathbf{h})}$$ $$J = -\sum_{c=1}^{C}\mathbf{v'}_{c,j*}\mathbf{h} + C\cdot\log\sum_{j'}^{V}\exp(\mathbf{v'}_{j'}\mathbf{h})$$ This function $J$ is the one we wish to minimize, thus we must perform gradient descent to reach the minimium.

where $j^{*}$ is the index of the actual word from the truth vectors $\mathbf{w}_{A}$. First we will show how to update $\mathbf{W'}$, then proceed to show how to update $\mathbf{W}$. We recall that using back propogation, we must take our loss function $J$ and compute the partial derivative with respect to $\mathbf{W'}$. Thus our partial is defined as $$\frac{\partial{J}}{\partial{\mathbf{W'}}} = \frac{\partial{J}}{\partial{\mathbf{o}_{c,j}}}\cdot{\frac{\partial{\mathbf{o}_{c,j}}}{\partial{\mathbf{W'}}}}\quad\quad   (1) $$ 

Taking the partial derivative of $(1)$ with respect to the net input of the $j$-th unit of every panel, we obtain $$\frac{\partial{J}}{\partial{\mathbf{o}_{c,j}}} = \mathbf{y}_{c,j} - \mathbf{t}_{c,j} := \mathbf{e}_{c,j} $$ where $\mathbf{t}_{c,j} = \mathbb{1}$ when element $j$ is equal to $j^{*}$, and $0$ for all other $j$ (also known as the indicator function). This is equivalent to taking the difference between the output layer panels $\mathbf{o}_{c,j}$ and their correspond truth vector $\mathbf{w}_{A}$. The term $\mathbf{y}_{c,j}$ is our softmax function from above. We note that this is the prediction error per unit. Thus, we must take the sum of all panels $\mathbf{o}_{c,j}$ for each unit $j$ with corresponding panel $c=1\ldots C$. We create a new vector $\mathbf{EI} = \{EI_{1}, EI_{2},\ldots,EI_{V} \}$ as the sum of the prediction error over all context words, meaning $\mathbf{EI}_{j} = \sum_{c=1}^{C}\mathbf{e}_{c,j}$. The second partial in $(1)$ becomes $\mathbf{h}_{i}$ where $i$ is the row of the hidden layer. Thus our new partial term becomes $$\frac{\partial{J}}{\partial{\mathbf{W'}_{i,j}}} = \sum_{c=1}^{C}\frac{\partial{J}}{\partial{\mathbf{o}_{c,j}}}\cdot{\frac{\partial{\mathbf{o}_{c,j}}}{\partial{\mathbf{W'}_{i,j}}}} = \mathbf{EI}_{j}\cdot \mathbf{h}_{i} $$

With this, our update equation for $\mathbf{W'}$ is defined as $$\mathbf{W'}_{i,j}^{(new)} = \mathbf{W'}_{i,j}^{(old)} - \nu\cdot\mathbf{EI}_{j}\cdot\mathbf{h}_{i} $$ 

where $\nu$ is the learning rate for the steepest gradient descent.

We now look to how to update our weight matrix $\mathbf{W}$, which is also thought of as our embedding matrix. Like above, we first take the partial derivative of our loss function $\mathbf{J}$ with respect to our hidden layer $\mathbf{h}_{i}$. This gives $$\frac{\partial{J}}{\partial{\mathbf{h}_{i}}}=\sum_{j=1}^{V}\frac{\partial{J}}{\partial{\mathbf{o}_{j}}}\cdot\frac{\partial{\mathbf{o}_{j}}}{\partial{\mathbf{h}_{i}}} = \sum_{j=1}^{V}\mathbf{e}_{j}\cdot\mathbf{W'}_{i,j} = \mathbf{EH}_{i}$$ where $\mathbf{EH}_{i}$ is the sum of all context word prediction entries for our vocabulary multipled by their weighted prediction error. The dimension of $\mathbf{EH}$ corresponds to the dimension of our space we are embedding into d. Just for clarity, the term $\mathbf{e}_{j} = \mathbf{y}_{j} - \mathbf{t}_{j}$ is the prediction error for the j-th word in the output layer.

Now, we can take the partial of our loss function $J$ with respect to input weight matrix $\mathbf{W}$ resulting in $$\frac{\partial{J}}{\partial{\mathbf{W}_{wt,}}$$